In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install spacy
!pip install spacy_transformers

In [1]:
import spacy

In [3]:
from spacy.tokens import DocBin
from tqdm import tqdm
import json


In [ ]:
!nvidia-smi

Sun Jan 14 04:10:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
# cv_data = json.load(open('/content/drive/MyDrive/Minor_Project/dataset/dataset.json'))
cv_data = json.load(open(r'C:\Users\deene\OneDrive\Documents\Minor_Project\dataset\dataset.json'))




In [8]:
len(cv_data)

1010

In [9]:
cv_data[0]

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [ ]:
 !python -m spacy init fill-config /content/drive/MyDrive/Minor_Project/config/base_config.cfg /content/drive/MyDrive/Minor_Project/config/config.cfg

2024-01-14 04:10:47.605676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 04:10:47.605737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 04:10:47.607334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 04:10:49.030556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Minor_Project/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spac

In [ ]:
# def get_spacy_doc(file,data):
#     nlp=spacy.blank("en")
#     db=DocBin()

#     for text, annot in tqdm(data):
#         doc=nlp.make_doc(text)
#         annot = annot["entities"]

#         ents=[]
#         entity_indices=[]
#         for start,end,label in annot:
#             skip_entity=False
#             for idx in range(start,end):
#                 if idx in entity_indices:
#                     skip_entity=True
#                     break
#             if skip_entity==True:
#                 continue
#             entity_indices = entity_indices+list(range(start,end))

#             try:
#                 span = doc.char_span(start,end,label=label , alignment ="strict")
#             except:
#                 continue
#             if span is None:
#                 err_data= str([start,end]) + "   "+str(text) + "\n"
#                 file.write(err_data)
#             else:
#                 ents.append(span)
#         try:
#             doc.ents = ents
#             db.add(doc)
#         except:
#             pass
#     return db

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot["entities"]

        ents = []
        entity_indices = []
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue
            entity_indices += list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label)
            except ValueError:
                continue
            if span is None:
                err_data = str([start, end]) + "   " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)
        try:
            doc.ents = ents
            db.add(doc)
        except ValueError:
            pass
    return db


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size = 0.2, random_state=42)

In [ ]:
len(train),len(test)

(811, 203)

In [ ]:
file_train = open('/content/drive/MyDrive/Minor_Project/model/train_file.txt', 'w')
db_train = get_spacy_doc(file_train, train)
db_train.to_disk('/content/drive/MyDrive/Minor_Project/model/train_data.spacy')
file_train.close()

file_test = open('/content/drive/MyDrive/Minor_Project/model/test_file.txt', 'w')
db_test = get_spacy_doc(file_test, test)
db_test.to_disk('/content/drive/MyDrive/Minor_Project/model/test_data.spacy')
file_test.close()


100%|██████████| 203/203 [00:01<00:00, 122.89it/s]


In [ ]:
!python -m spacy train /content/drive/MyDrive/Minor_Project/config/config.cfg --output /content/drive/MyDrive/Minor_Project/model/output80 --paths.train /content/drive/MyDrive/Minor_Project/model/train_data.spacy --paths.dev /content/drive/MyDrive/Minor_Project/model/test_data.spacy --gpu-id 0

2024-01-14 04:11:08.169600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 04:11:08.169656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 04:11:08.170943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 04:11:09.273437: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/Minor_Project/model/output80
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model 

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/Minor_Project/model/output/model-best')

In [ ]:
 !pip install PyMuPDF

In [ ]:
import sys,fitz
fname = '/content/drive/MyDrive/Minor_Project/test/test2.pdf'
doc = fitz.open(fname)

In [ ]:
text =" "
for page in doc:
  text = text + str(page.get_text())


In [ ]:
text

' John Doe \n123 Main Street, Cityville, State 12345 \njohn.doe@email.com | (555) 123-4567 \n \nObjective: \nEnthusiastic and skilled Full Stack Developer with a passion for creating efficient and scalable web \napplications. Adept at both front-end and back-end development, with a strong foundation in various \nprogramming languages and frameworks. Seeking a challenging role to contribute technical expertise \nand creativity to innovative projects. \n \nEducation: \nBachelor of Science in Computer Science \nTech University, Cityville, State \nSeptember 2018 - May 2022 \n \nSkills: \n \n**Programming Languages:** \n- JavaScript (Node.js, React.js) \n- HTML5, CSS3, SASS/LESS \n- Python (Django, Flask) \n \n**Database Systems:** \n- MySQL, PostgreSQL, MongoDB \n \n**Server Management:** \n- AWS, Heroku \n \n**Version Control:** \n- Git, GitHub \n \n**Development Tools:** \n- Visual Studio Code, Sublime Text, Atom \n \n**Web Development:** \n- Responsive Web Design \n- RESTful API Develop

In [ ]:
doc= nlp(text)
for ent in doc.ents:
  print(ent.text," --->",ent.label_)

John Doe  ---> NAME
john.doe@email.com  ---> EMAIL ADDRESS
Bachelor of Science in Computer Science  ---> DEGREE
Tech University, Cityville, State  ---> UNIVERSITY
HTML5  ---> SKILLS
CSS3  ---> SKILLS
SASS/LESS  ---> SKILLS
Junior Full Stack Developer  ---> WORKED AS
Task Management App | January 2021 - May 2021:** 
- Led the development of a task management application using the MEAN (MongoDB, Express.js, 
Angular, Node.js) stack. 
- Implemented real-time updates using WebSockets for collaborative task tracking. 
- Utilized Agile methodologies for project management, ensuring timely delivery of features. 
 
Certifications: 
 
- Full Stack Web Development Certification | Tech University, 2022 
- AWS Certified Developer – Associate | 2023 
 
References: 
Available upon request. 
  ---> WORKED AS
